In [1]:
import numpy as np
import regex as re

Firstly, we need to create methods that implement basic functions of the tableau

# Tableau

In [2]:
def create_tableau(c, A, b):
  n, m = A.shape

  I = np.identity(n)
  zeros = np.zeros((1,n))
  
  vero = np.append(I, zeros, axis=0)
  A_ext = np.append(A, I, axis=1)
  b_ext = np.append(b, [[0]], axis=0)
  c_ext = np.append(c, zeros, axis=1)

  tableau = np.append(A_ext, -c_ext, axis=0)
  tableau = np.append(tableau, b_ext, axis=1)
  tableau = np.append(vero, tableau, axis=1)

  return tableau

In [3]:
def FPI(tableau, n, m):  
  neg = tableau[:n,n+m+n] < 0
  for i in range(n):
    if neg[i]:
      tableau[i,:] *= -1
  return tableau

In [4]:
def checkUnlimited(tableau, n, m):
  for i in range(n, n+m+n+1):
    if tableau[n,i] < 0 and all(tableau[:n,i] <= 0):
      return True
  return False

In [5]:
def checkUnfeasable(tableau, n, m):
  for i in range(n):
    if tableau[i, n+m+n] > 0 and all(tableau[i, n:(n+m+n)] <= 0):
      return True
  return False

def UnfeasableCert(tableau, n, m):
  y = np.zeros((1,n))
  for i in range(n):
    if tableau[i, n+m+n] > 0 and all(tableau[i, n:(n+m+n)] <= 0):
      y = -tableau[i,:n]
      break
  return y

In [6]:
def checkOptimal(tableau, n, m):
  return all(tableau[n,:(n+m+n)] >= 0)

In [7]:
def get_pivot(tableau, n, m):
  # getting better result
  min_c = min(tableau[n,n:(n+m+n)])

  col = -1
  row = -1
  if min_c < 0:
    col = n
    while col < n+m+n:
      if tableau[n,col] == min_c:
        break
      col+=1
    
    min_r = np.inf
    for i in range(n):
      if tableau[i,col] > 0 and tableau[i,(n+m+n)]/tableau[i,col] < min_r:
        row = i
        min_r = tableau[i,(n+m+n)]/tableau[i,col]

  return row, col

def aux_pivot(tableau, n, m):
  row = -1
  col = -1
  for i in range(n):
    unsolved_row = tableau[i,(n+m+n)] != 0
    temp_col = -1
    j = n
    while j < n+m+n and unsolved_row:
      if tableau[i,j] > 0:
        temp_col = j

        if np.count_nonzero(tableau[:n,j]) == 1:
          unsolved_row = False
      j += 1

    if unsolved_row:
      row = i
      col = temp_col
      break

  return row, col

def pivot(tableau, pos):
  r, c = pos
  
  tableau[r,:] /= tableau[r,c]
  operation = np.identity(tableau.shape[0])
  operation[:,r] += -tableau[:,c]
  operation[r,r] = 1

  tableau = np.round(operation @ tableau, decimals=6)
  return operation

In [8]:
def has_obvious_Sol(tableau,n,m):
  solved = (tableau[:n,(n+m+n)] == 0).tolist()

  for i in range(n):
    for j in range(n,n+m+n):
      if tableau[i,j] > 0 and np.count_nonzero(tableau[:n,j]) == 1:
        tableau[i,:] /= tableau[i,j]
        solved[i] = True
        break
  
  tableau = np.round(tableau, decimals=6)
  return all(solved)

In [9]:
def obvious_Sol(tableau, n, m):
  sol = np.zeros(m)
  if not has_obvious_Sol(tableau,n,m):
    return sol

  basic_col = []
  basic_row = []
  unsolved = (tableau[:n,(n+m+n)] != 0).tolist()

  j=n
  while (j < n+m and any(unsolved)):
    if tableau[n,j] == 0  and np.count_nonzero(tableau[:n,j]) == 1:
      for i in range(n):
        if tableau[i,j] == 1 and unsolved[i]:
          basic_row.append(i)
          basic_col.append(j-n)
          break
    j+=1
  
  for i in range(len(basic_col)):
    sol[basic_col[i]] = tableau[basic_row[i],(n+m+n)]
  
  return np.array([sol])

In [10]:
def format(array):
  return re.sub('( \[|\[|\])', '', str(array))

def print_step(pos, piv):
  print('Tableau Step')
  print('Pivoting pos = ', pos)
  print('Operantion Matrix')
  print(piv)

def print_tableau(tableau):
  print('Current Tableau')
  print(tableau)

In [11]:
def simplex(c,A,b):
  n, m = A.shape
  tableau = create_tableau(c,A,b)
  tableau = FPI(tableau, n, m)
  # print_tableau(tableau)

  # get obvious solution
  i=0
  while not has_obvious_Sol(tableau, n, m):
    if checkUnfeasable(tableau, n, m):
      y = UnfeasableCert(tableau, n, m)
      print('inviavel')
      print(format(y))
      return

    pos = aux_pivot(tableau, n, m)
    p = pivot(tableau, pos)
    # print_step(pos, p)

    tableau = p @ tableau
    tableau = FPI(tableau, n, m)
    # print_tableau(tableau)

    i+=1
    if i>10: break
  
  # get optimal sol
  while not checkOptimal(tableau, n, m):
    if checkUnlimited(tableau, n, m):
      sol = obvious_Sol(tableau, n, m)
      print('ilimitada')
      print(format(sol))
      print(format(sol))
      return
    
    pos = get_pivot(tableau, n, m)
    p = pivot(tableau, pos)
    # print_step(pos, p)

    tableau = p @ tableau
    # print_tableau(tableau)

  sol = obvious_Sol(tableau, n, m)
  y = tableau[n,:n]
  print('otima')
  print(format(tableau[n,(n+m+n)]))
  print(format(sol))
  print(format(y))

Honestly, this was easier than I expected, probably won't work. And finally...

# Reading Input

In [17]:
n, m = map(int, input().split())
c = np.array(list(map(int, input().split())), dtype=float).reshape(1,m)

A = []
b = []
for i in range(n):
  entries = list(map(int, input().split()))
  A.append(entries[:m])
  b.append(entries[m])
A = np.array(A, dtype=float).reshape(n,m)
b = np.array(b, dtype=float).reshape(n,1)

7 5
-7 -2 -2 5 -2
2 0 -1 1 2 -9
2 -2 -1 0 1 6
0 2 0 -2 -1 5
-1 1 1 -1 -2 -3
0 -1 -1 0 2 8
0 0 1 0 1 7
1 0 -2 -2 1 -10


# Runing

In [18]:
simplex(c,A,b)

inviavel
 1. -0. -0.  1. -0. -0. -0.
